<a href="https://colab.research.google.com/github/cheitra07/Finetune_Search_excel/blob/main/search_tools_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers datasets scikit-learn

In [2]:
from google.colab import files
uploaded = files.upload()


Saving dsfinetune.csv to dsfinetune (1).csv


In [27]:
# Step 1: Install dependencies
!pip install -q transformers datasets

# Step 2: Load dataset
import pandas as pd

data = {
    "text": [
        "Engine noise on startup",
        "Screen not turning on",
        "Oil leakage from valve",
        "Engine makes rattling noise",
        "Display flickers randomly",
        "Oil leak near gasket",
    ],
    "label": [
        "engine_issue",
        "display_issue",
        "oil_leak",
        "engine_issue",
        "display_issue",
        "oil_leak"
    ]
}

df = pd.DataFrame(data)

# Step 3: Encode labels
label2id = {"engine_issue": 0, "display_issue": 1, "oil_leak": 2}
df["label"] = df["label"].map(label2id)

# Step 4: Train-test split
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.33, random_state=42)

# Step 5: Load tokenizer and model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Step 6: Tokenize datasets
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

train_dataset = train_dataset.map(tokenize)
test_dataset = test_dataset.map(tokenize)

# Step 7: Define metrics
from sklearn.metrics import accuracy_score
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    return {"accuracy": accuracy_score(p.label_ids, preds)}

# Step 8: Set training arguments
from transformers import TrainingArguments, Trainer
import os
os.environ["WANDB_DISABLED"] = "true"  # disables wandb

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=5,
    logging_strategy="epoch",
    save_strategy="no"
)

# Step 9: Train
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

# Step 10: Test prediction
text = "Oil leak found under machine"
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
outputs = model(**inputs)
predicted = outputs.logits.argmax().item()
id2label = {v: k for k, v in label2id.items()}
print(f"Predicted label: {id2label[predicted]}")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-27-1256669008.py:77: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
2,1.202700
4,1.106000
6,1.023900
8,0.877100
10,0.858500


Predicted label: oil_leak


In [28]:
model_path = "./fine_tuned_distilbert"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)


('./fine_tuned_distilbert/tokenizer_config.json',
 './fine_tuned_distilbert/special_tokens_map.json',
 './fine_tuned_distilbert/vocab.txt',
 './fine_tuned_distilbert/added_tokens.json',
 './fine_tuned_distilbert/tokenizer.json')

In [29]:
import shutil
shutil.make_archive("fine_tuned_distilbert", 'zip', model_path)


'/content/fine_tuned_distilbert.zip'

In [30]:
from google.colab import files
files.download("fine_tuned_distilbert.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>